# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [2]:
!pip install pyorient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyorient: filename=pyorient-1.5.5-py3-none-any.whl size=79022 sha256=22dddb21f45d2b2a0e97a07b0e80ea60bf07591dd0bfbd9c6beefa60cf9561db
  Stored in directory: /home/jovyan/.cache/pip/wheels/a3/08/60/10d34e8571e6eb7c98ccbeff65f2d4e220d89d568f31bf88d4
Successfully built pyorient


In [3]:
import pyorient

In [4]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("orientdb", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [5]:
print(client.db_list())

{{'databases': {'gods': 'memory:gods'}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [7]:
db_name = "gods"
db_type = pyorient.DB_TYPE_GRAPH
storage_type = pyorient.STORAGE_TYPE_MEMORY

In [ ]:
client.db_create(db_name, db_type, storage_type)

**Q:** Connect your pyorient client to the `gods` database.

In [8]:
client.db_open(db_name, "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
client.command("CREATE CLASS Greekdeities EXTENDS V")

PyOrientSchemaException: com.orientechnologies.orient.core.exception.OSchemaException - Class 'Greekdeities' already exists in current database
	DB name="gods"

In [7]:


vertex_content = {
    "name": "Zeus",
    "symbol": "thunder"
}

result = client.command(f"CREATE VERTEX Greekdeities CONTENT {vertex_content}")

print(result)

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [9]:
query = "SELECT * FROM V"
result = client.query(query)

# Print the result
for record in result:
    print(record)

{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17e021ec90>, 'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17e021f710>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f17c6375bd0>, 'name': 'Zeus', 'symbol': 'thunder'},'version':27,'rid':'#33:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17c6376fd0>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17c6377110>, 'name': 'Héra', 'symbol': 'tiara'},'version':10,'rid':'#34:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17c6380610>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17c63805d0>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f17c63808d0>, 'name': 'Poséidon', 'symbol': 'trident'},'version':15,'rid':'#35:0'}
{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17c6380c10>, 'name': 'Athe

**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [24]:
vertices_content = [
    {"name": "Héra", "symbol": "tiara"},
    {"name": "Poséidon", "symbol": "trident"},
    {"name": "Athena", "symbol": "helmet"},
    {"name": "Arès", "symbol": "weapons"}
]

for content in vertices_content:
    query = f"CREATE VERTEX Greekdeities CONTENT {content}"
    result = client.command(query)

**Q:** Display all vertices with name = `Arès`

In [9]:
query = "SELECT * FROM V WHERE name = 'Arès'"
result = client.query(query)

for record in result:
    print(record)

{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e858ed0>, 'name': 'Arès', 'symbol': 'weapons'},'version':8,'rid':'#37:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
client.command("CREATE CLASS relationship EXTENDS E")

PyOrientSchemaException: com.orientechnologies.orient.core.exception.OSchemaException - Class 'relationship' already exists in current database
	DB name="gods"

In [61]:

edge_content = {
    "kind": "sibling"
}

create_edge_query = f"CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Zeus') TO (SELECT * FROM V WHERE name = 'Poséidon') CONTENT {edge_content}"
result_edge = client.command(create_edge_query)

**Q:** Redisplay all vertices, discuss.

In [11]:
query = "SELECT * FROM V"
result = client.query(query)

for record in result:
    print(record)

{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f8e90>, 'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f9050>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f96d0>, 'name': 'Zeus', 'symbol': 'thunder'},'version':27,'rid':'#33:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f1c90>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f1110>, 'name': 'Héra', 'symbol': 'tiara'},'version':10,'rid':'#34:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f3290>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f2150>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f3a10>, 'name': 'Poséidon', 'symbol': 'trident'},'version':15,'rid':'#35:0'}
{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e8f2e50>, 'name': 'Athe

**Q:** Display all edges. They are contained in the class `E`

In [33]:
query = "SELECT * FROM E"
result = client.query(query)

for record in result:
    print(record)

{'@relationship':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e462990>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e4632d0>},'version':1,'rid':'#45:0'}
{'@relationship':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e460490>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e461310>},'version':1,'rid':'#46:0'}
{'@relationship':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e461a10>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e462410>},'version':1,'rid':'#47:0'}
{'@relationship':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e462190>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e4626d0>},'version':1,'rid':'#48:0'}
{'@relationship':{'kind': 'mother', 'out': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e462c10>, 'in': <pyorient.otypes.OrientRecordLink object at 0x7f1e4e462350>},'version':1,'rid':'#49:0'}
{'@relat

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [64]:
def create_edge(name1, name2, edge_content):
    create_edge_query = f"CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = '{name1}') TO (SELECT * FROM V WHERE name = '{name2}') CONTENT {edge_content}"
    print(create_edge_query)
    result_edge = client.command(create_edge_query)


In [65]:
edge_create = [
    {"kind": "sibling"},
    {"kind": "father"},
    {"kind": "father"},
    {"kind": "mother"},
    {"kind": "sibling"},
    {"kind": "sibling"}
]
name1 = ['Zeus', 'Zeus', 'Zeus', 'Héra', 'Héra', 'Poséidon']
name2 = ['Héra', 'Arès', 'Athena', 'Arès', 'Zeus', 'Zeus']

In [49]:
name1[i]

'Zeus'

In [66]:
for i in range(len(name1)):
    create_edge(name1[i], name2[i], edge_create[i])

CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Zeus') TO (SELECT * FROM V WHERE name = 'Héra') CONTENT {'kind': 'sibling'}
CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Zeus') TO (SELECT * FROM V WHERE name = 'Arès') CONTENT {'kind': 'father'}
CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Zeus') TO (SELECT * FROM V WHERE name = 'Athena') CONTENT {'kind': 'father'}
CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Héra') TO (SELECT * FROM V WHERE name = 'Arès') CONTENT {'kind': 'mother'}
CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Héra') TO (SELECT * FROM V WHERE name = 'Zeus') CONTENT {'kind': 'sibling'}
CREATE EDGE relationship FROM (SELECT * FROM V WHERE name = 'Poséidon') TO (SELECT * FROM V WHERE name = 'Zeus') CONTENT {'kind': 'sibling'}


### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [23]:
zeus_vertex = client.query("SELECT * FROM V WHERE name = 'Zeus'")[0]

query_outgoing = f"SELECT EXPAND(out()) FROM {zeus_vertex._rid} LIMIT -1"
query_incoming = f"SELECT EXPAND(in()) FROM {zeus_vertex._rid} LIMIT -1"

result_outgoing = client.query(query_outgoing)
result_incoming = client.query(query_incoming)

result_both = result_outgoing + result_incoming

for record in result_both:
    print(record)

{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e709c50>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e7094de10>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f1e7094f1d0>, 'name': 'Poséidon', 'symbol': 'trident'},'version':15,'rid':'#35:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e581a50>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e583550>, 'name': 'Héra', 'symbol': 'tiara'},'version':10,'rid':'#34:0'}
{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e582250>, 'name': 'Arès', 'symbol': 'weapons'},'version':8,'rid':'#37:0'}
{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e580d10>, 'name': 'Athena', 'symbol': 'helmet'},'version':4,'rid':'#36:0'}
{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e580bd0>, 'i

**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [41]:
query_all_incoming_father = "SELECT * FROM V WHERE @rid IN (SELECT EXPAND(in) FROM E WHERE kind = 'father' LIMIT -1)"
result_all_incoming_father = client.query(query_all_incoming_father)

for record in result_all_incoming_father:
    print(record)


{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e454210>, 'name': 'Athena', 'symbol': 'helmet'},'version':4,'rid':'#36:0'}
{'@Greekdeities':{'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e454690>, 'name': 'Arès', 'symbol': 'weapons'},'version':8,'rid':'#37:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [42]:
query_all_outgoing_mother = "SELECT * FROM V WHERE @rid IN (SELECT EXPAND(out) FROM E WHERE kind = 'mother' LIMIT -1)"
result_all_outgoing_mother = client.query(query_all_outgoing_mother)

for record in result_all_outgoing_mother:
    print(record)

{'@Greekdeities':{'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e70950750>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f1e4e7bb210>, 'name': 'Héra', 'symbol': 'tiara'},'version':10,'rid':'#34:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [16]:
query_siblings = "SELECT * FROM V WHERE name != 'Zeus' AND @rid IN (SELECT EXPAND(in) FROM E WHERE kind = 'sibling' LIMIT -1)"
result_siblings = client.query(query_siblings)

for record in result_siblings:
    print(record)

{'@Greekdeities':{'name': 'Héra', 'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17dc049e10>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17e0230690>, 'symbol': 'tiara'},'version':10,'rid':'#34:0'}
{'@Greekdeities':{'name': 'Poséidon', 'out_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17e0232610>, 'in_relationship': <pyorient.otypes.OrientBinaryObject object at 0x7f17e02324d0>, 'in_': <pyorient.otypes.OrientBinaryObject object at 0x7f17e0231310>, 'symbol': 'trident'},'version':15,'rid':'#35:0'}


In [41]:
query_siblings = "SELECT EXPAND(in) FROM E WHERE kind = 'sibling' AND out IN (SELECT * FROM V name = 'Zeus')"
result_siblings = client.query(query_siblings)

for record in result_siblings:
    print(record)

PyOrientSQLParsingException: com.orientechnologies.orient.core.sql.OCommandSQLParsingException - Error parsing query:
SELECT EXPAND(in) FROM E WHERE kind = 'sibling' AND IN (SELECT * FROM V name = 'Zeus')
                                                     ^
Encountered "" at line 1, column 53.
Was expecting one of:
    
	DB name="gods"

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

**Q:** Create the following people: `peter`, `meredith`.

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


**Q:** Who bought Bolognese sauce?

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.